In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

silver_schema = "workspace.stock_project"
gold_schema = "workspace.stock_project"


**DAILY STOCK METRICS**

In [0]:
df_stock = spark.table(f"{silver_schema}.silver_stock_prices")

w = Window.partitionBy("ticker").orderBy("date")

df_gold_stock_metrics = (
    df_stock
    .withColumn("prev_close", F.lag("close").over(w))
    .withColumn(
        "daily_return",
        F.try_divide(F.col("close") - F.col("prev_close"), F.col("prev_close"))
    )
    .withColumn("ma_7", F.avg("close").over(w.rowsBetween(-6, 0)))
    .withColumn("ma_30", F.avg("close").over(w.rowsBetween(-29, 0)))
    .withColumn("volatility_7", F.stddev("daily_return").over(w.rowsBetween(-6, 0)))
    .withColumn("daily_range", F.col("high") - F.col("low"))
    .drop("prev_close")
)

df_gold_stock_metrics.write.partitionBy("ticker").mode("overwrite").saveAsTable(
    f"{gold_schema}.gold_daily_stock_metrics"
)

display(df_gold_stock_metrics.limit(10 ))

date,ticker,open,close,volume,high,low,daily_return,ma_7,ma_30,volatility_7,daily_range
2021-01-01,AAPL,141.77,141.91,4852219,144.59,141.34,null,141.91,141.91,null,3.25
2021-01-02,AAPL,146.01,146.34,1232267,148.04,144.67,0.03121696850116276,144.125,144.125,null,3.3700000000000045
2021-01-03,AAPL,143.79,142.75,3742555,143.88,140.75,-0.024531911985786546,143.66666666666666,143.66666666666666,0.039420411435880244,3.1299999999999955
2021-01-04,AAPL,141.6,142.35,450325,142.98,140.16,-0.0028021015761821766,143.3375,143.3375,0.028099286660319366,2.819999999999993
2021-01-05,AAPL,146.26,144.26,2330027,146.95,142.91,0.013417632595714764,143.522,143.522,0.02373022603863783,4.039999999999992
2021-01-06,AAPL,146.2,143.06,4916393,149.05,142.72,-0.008318314154997843,143.44500000000002,143.44500000000002,0.021314646192024517,6.3300000000000125
2021-01-07,AAPL,146.45,149.37,4811653,150.61,145.35,0.04410736753809592,144.29142857142858,144.29142857142858,0.02572586645843398,5.260000000000019
2021-01-08,AAPL,142.14,142.09,3847122,143.8,142.01,-0.04873803307223673,144.31714285714287,144.01625,0.03201963809417726,1.7900000000000205
2021-01-09,AAPL,142.13,141.36,4887975,142.87,140.0,-0.005137588852135898,143.6057142857143,143.72111111111113,0.02903971868960013,2.8700000000000045
2021-01-12,AAPL,139.48,138.14,3231031,142.31,137.22,-0.022778720996038674,142.94714285714286,143.163,0.02884579272967559,5.090000000000003


**PORTFOLIO HOLDINGS**

In [0]:
df_port = spark.table(f"{silver_schema}.silver_portfolio")
df_stock = spark.table(f"{silver_schema}.silver_stock_prices")

# Net quantity per date
df_port_net = (
    df_port.groupBy("ticker", "date")
    .agg(
        F.sum(
            F.when(F.col("action") == "BUY", F.col("quantity"))
             .when(F.col("action") == "SELL", -F.col("quantity"))
             .otherwise(0)
        ).alias("net_quantity")
    )
)

# Rolling quantity held
w = Window.partitionBy("ticker").orderBy("date")
df_holdings = (
    df_port_net
    .withColumn("quantity_held", F.sum("net_quantity").over(w))
    .withColumn(
        "quantity_held",
        F.when(F.col("quantity_held") < 0, 0)
         .otherwise(F.col("quantity_held"))
    )
)

# Join with stock close price for market value
df_gold_holdings = (
    df_holdings
    .join(df_stock.select("ticker", "date", "close"), ["ticker", "date"], "left")
    .withColumn("market_value", F.col("quantity_held") * F.col("close"))
)

df_gold_holdings.write.partitionBy("date").mode("overwrite").format("delta").saveAsTable(
    f"{gold_schema}.gold_portfolio_holdings"
)

display(df_gold_holdings.limit(10))

ticker,date,net_quantity,quantity_held,close,market_value
AAPL,2021-01-01,-273,0,141.91,0.0
AAPL,2021-01-02,446,173,146.34,25316.82
AAPL,2021-01-05,-295,0,144.26,0.0
AAPL,2021-01-10,84,0,null,null
AAPL,2021-01-11,-334,0,null,null
AAPL,2021-01-12,-191,0,138.14,0.0
AAPL,2021-01-13,52,0,137.62,0.0
AAPL,2021-01-15,529,18,135.06,2431.08
AAPL,2021-01-16,-232,0,143.82,0.0
AAPL,2021-01-17,175,0,150.56,0.0


**SECTOR PERFORMANCE**

In [0]:

df_company = spark.table(f"{silver_schema}.silver_company_sector")
df_stock_metrics = spark.table(f"{gold_schema}.gold_daily_stock_metrics")

df_gold_sector = (
    df_stock_metrics
    .join(df_company, "ticker", "left")
    .groupBy("sector", "date")
    .agg(
        F.avg("daily_return").alias("avg_sector_return"),
        F.stddev("daily_return").alias("sector_volatility"),
        F.sum("volume").alias("sector_volume")
    )
)

df_gold_sector.write.mode("overwrite").format("delta").saveAsTable(
    f"{gold_schema}.gold_sector_performance"
)

display(df_gold_sector.limit(10))

sector,date,avg_sector_return,sector_volatility,sector_volume
Finance,2024-04-29,0.005959343871900766,0.03279504165625567,14721876
Automotive,2022-05-01,-0.005637685089384055,0.01620580832071611,22209586
Automotive,2024-06-08,-0.02060228220719539,0.014401989422371883,8585233
Finance,2023-06-13,0.003170600860919552,0.0150347493394178,25938058
Finance,2024-06-27,0.0167538909912239,0.020376659227511577,12507730
Retail,2021-01-27,0.016613548840905618,0.02071191412049135,5415559
Retail,2022-09-20,0.004178317609658199,0.011286537719412081,17670222
Retail,2024-08-23,-0.004770387505580621,0.030276767968896855,9689814
Automotive,2021-04-21,0.014436691257315228,0.030918327077427164,7466546
Retail,2023-05-24,-0.00525384948793418,0.019529104103387797,20862510


**PORTFOLIO TRADES FACT**

In [0]:
df_port = spark.table(f"{silver_schema}.silver_portfolio")

df_gold_trades = df_port.withColumn(
    "trade_value", F.col("quantity") * F.col("price")
)

df_gold_trades.write.mode("overwrite").format("delta").saveAsTable(
    f"{gold_schema}.gold_portfolio_trades_fact"
)

display(df_gold_trades.limit(10))

transaction_id,date,ticker,action,quantity,price,trade_value
1,2022-09-17,PFE,SELL,294,223.2,65620.8
2,2022-02-25,AMZN,SELL,168,284.82,47849.76
3,2021-04-23,TGT,SELL,463,459.46,212729.97999999998
4,2021-02-16,PEP,BUY,482,70.69,34072.58
5,2021-07-01,BAC,SELL,344,310.09,106670.95999999999
6,2022-09-21,INTC,SELL,365,259.81,94830.65
7,2021-09-21,META,BUY,179,265.47,47519.130000000005
8,2022-03-16,AVGO,SELL,259,373.96,96855.64
9,2021-05-10,AVGO,SELL,486,157.84,76710.24
11,2021-11-09,JPM,BUY,416,499.7,207875.19999999998


**BENCHMARK COMPARISION**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df_bench = spark.table(f"{silver_schema}.silver_benchmark_index")
df_holdings = spark.table(f"{gold_schema}.gold_portfolio_holdings")

#----------------------------------------------
# 1. Compute Benchmark Daily Return
#----------------------------------------------
df_bench_keyed = df_bench.withColumn("key", F.lit(1))
w_bench = Window.partitionBy("key").orderBy("date")
df_bench2 = (
    df_bench_keyed
    .withColumn("prev_close", F.lag("close").over(w_bench))
    .withColumn(
        "benchmark_return",
        F.try_divide(
            F.col("close") - F.col("prev_close"),
            F.col("prev_close")
        )
    )
    .drop("key")
)

#----------------------------------------------
# 2. Compute Portfolio Daily Return (safe)
#----------------------------------------------
df_hold_keyed = df_holdings.withColumn("key", F.lit(1))
w_port = Window.partitionBy("key").orderBy("date")
df_port_ret = (
    df_hold_keyed
    .groupBy("date", "key")
    .agg(F.sum("market_value").alias("portfolio_value"))
    .withColumn("prev_value", F.lag("portfolio_value").over(w_port))
    .withColumn(
        "portfolio_return",
        F.try_divide(
            F.col("portfolio_value") - F.col("prev_value"),
            F.col("prev_value")
        )
    )
    .drop("key")
)

#----------------------------------------------
# 3. Join Portfolio vs Benchmark
#----------------------------------------------
df_gold_compare = (
    df_port_ret
    .join(df_bench2.select("date", "benchmark_return"), "date", "left")
    .withColumn(
        "excess_return",
        F.col("portfolio_return") - F.col("benchmark_return")
    )
)

df_gold_compare.write.mode("overwrite").format("delta").saveAsTable(
    f"{gold_schema}.gold_benchmark_comparison"
)

display(df_gold_compare.limit(10))

date,portfolio_value,prev_value,portfolio_return,benchmark_return,excess_return
2021-01-01,543812.15,null,null,null,null
2021-01-02,619630.7999999999,543812.15,0.13942066207972717,0.002013963862594727,0.13740669821713244
2021-01-03,753405.1000000001,619630.7999999999,0.21589356113350108,0.006046805171766788,0.2098467559617343
2021-01-04,456384.41000000003,753405.1000000001,-0.3942376949664928,0.0030801140489936196,-0.3973178090154864
2021-01-05,336673.79,456384.41000000003,-0.2623021675959528,0.0062201913948384605,-0.2685223589907913
2021-01-06,1001499.6500000001,336673.79,1.9746884959473683,0.005291434874755402,1.9693970610726128
2021-01-07,412829.31,1001499.6500000001,-0.5877888624324532,-0.001540085387012652,-0.5862487770454405
2021-01-08,962427.22,412829.31,1.3312957599837083,-0.002928723244021221,1.3342244832277295
2021-01-09,972387.8099999999,962427.22,0.010349447514587095,-0.001163740530620882,0.011513188045207977
2021-01-10,567763.15,972387.8099999999,-0.41611449242663784,0.002725093263726297,-0.4188395856903641


**RISK INDICATORS**

In [0]:
df_stock = spark.table(f"{gold_schema}.gold_daily_stock_metrics")

w = Window.partitionBy("ticker").orderBy("date")

df_gold_risk = (
    df_stock
    .withColumn(
        "volatility_30d",
        F.stddev("daily_return").over(w.rowsBetween(-29, 0))
    )
    .withColumn(
        "downside_deviation",
        F.sqrt(
            F.avg(
                F.when(F.col("daily_return") < 0, F.col("daily_return") ** 2)
                 .otherwise(0)
            ).over(w.rowsBetween(-29, 0))
        )
    )
    .withColumn(
        "rolling_sharpe",
        F.col("daily_return") / F.col("volatility_30d")
    )
)

df_gold_risk.write.mode("overwrite").format("delta").saveAsTable(
    f"{gold_schema}.gold_risk_indicators"
)

display(df_gold_risk.limit(10))

date,ticker,open,close,volume,high,low,daily_return,ma_7,ma_30,volatility_7,daily_range,volatility_30d,downside_deviation,rolling_sharpe
2021-01-01,GOOGL,261.07,259.3,2844268,262.91,258.4,null,259.3,259.3,null,4.510000000000048,null,0.0,null
2021-01-02,GOOGL,267.15,264.7,4009376,267.31,262.24,0.02082529888160423,262.0,262.0,null,5.069999999999993,null,0.0,null
2021-01-03,GOOGL,260.8,256.88,2749803,262.01,255.24,-0.029542878730638433,260.29333333333335,260.29333333333335,0.03561567994562523,6.769999999999982,0.03561567994562523,0.017056588987770568,-0.8294907966306359
2021-01-04,GOOGL,265.82,263.05,3634873,266.46,262.32,0.024018997197134913,260.9825,260.9825,0.03004448806301313,4.139999999999986,0.03004448806301313,0.014771439365319217,0.7994477105670501
2021-01-05,GOOGL,260.62,262.82,3910730,265.07,257.91,-8.743584869797307E-4,261.35,261.35,0.024712456234832404,7.159999999999968,0.024712456234832404,0.013217762189243398,-0.03538128620931316
2021-01-06,GOOGL,258.74,255.41,874561,260.05,253.05,-0.028194201354539216,260.36,260.36,0.025696097144579017,7.0,0.025696097144579017,0.0166756268071961,-1.0972172620575267
2021-01-07,GOOGL,260.3,256.8,4368067,262.57,255.0,0.005442230139775321,259.8514285714286,259.8514285714286,0.023225553388253307,7.569999999999993,0.023225553388253307,0.015438630474408127,0.2343207952378786
2021-01-08,GOOGL,263.79,261.4,739792,264.28,260.9,0.01791277258566965,260.15142857142854,260.045,0.02242178007076951,3.3799999999999955,0.02242178007076951,0.014441516439060612,0.7989005569197382
2021-01-09,GOOGL,265.93,264.35,655828,266.07,262.15,0.011285386381025424,260.1014285714286,260.5233333333333,0.02130421050237115,3.920000000000016,0.021052479187868497,0.013615592272902348,0.5360597334079605
2021-01-10,GOOGL,267.27,265.05,2657665,269.92,262.92,0.00264800453943631,261.2685714285714,260.976,0.01687685507299597,7.0,0.01969279532348237,0.01291688498236809,0.13446565080980344


In [0]:
dbutils.notebook.exit("SUCCESS")
